# Computer Assignment #2 Game Report
# AI course - Fall 1400
<h2>By Kianoush Arshi<br>Std ID: 810198438</h2>


<h2>Part 1: Genes and Chromosomes</h2>
In this algorithm we have 3 different classes of data:<br>
<li>Genome, which is the smallest and mine is each cell of the board.<br>
<li>Chromosme, which is a cluster of genes together and mine is a 9 in 9 board.<br>
<li>Population, which is a list of chromosmes and my population size is 200.<br></li>

<h2>Genome</h2>
<p>Each cell is a genome, in the algorithms mutation function we swap 2 genes in the same chromosome and in the cross over we swap a group of genes between two chromosomes.<br></p>

<h2>Chromosome</h2>
<p>
    Each chromosome is a possible solution to our problem.<br>
    We can find out if a solution is found using a fitness function which is defined as follows:<br>
    <li>Get the count of duplicate numbers in each column and divide them by (9*9)</li>
    <li>Get the count of duplicate numbers in each 3by3 block and divide them by (9*9)</li>
    <li>Check if there is any duplicates:
        <li>If there isn't any: set fitness to 1 and return.
        <li>If there is: return the multitude of block_sum and col_sum as fitness.
    </li>    
</p>

<h2>Population</h2>
<p>
    We have a ppulation size of 200. Initial population is generated randomly with the only two constraints of not having duplicated any of the fixed values in a block, row or column and not inputted more than 9 times.<br>
    Each generation, we take 20% of the top(fittesst) chromosomes and keep them as elites in the next generation. Using these elites, we create children by randomly crossing over two of them and possibly mutating the child.<br>
    <h4>Cross Over:</h4>
    <p>
        A child is created using a cross over between two randomly selected chromosomes and then using two randomly generated cross over points, switched genes between these points in each row.
        The aforementioned approach for cross over had some problems, only the elites had a chance to be a parent. This might cause the algorithm to get stuck in local minima. Thus a new approach is introduced:<br>
        Have a competition between two randomly chosen chromosomes and give the weaker one a chance to be chosen reducing the chance for the population to go stale.<br>
        Afterwards, the cross over is as follows:<br>
    <li>Randomly choose two cross over point.</li>
    <li>Apply cyclic cross over to rows from cross over point 1 to cross over point 2.</li>
    </p>
    <h4>Mutation:</h4>
    <p>
        In general, optimization solving algorithms converge to a local minimum.  To get out of this local minimum in a Genetic Algorithm, you can use mutations. The higher the mutation rate is, the more 'space' will be searched and the higher the chance that the global minimum is found. But if the mutation rate is too high, the algorithm won't converge anymore.<br>
        The cross overed child is then mutated with an adaptive mutation rate which chaanges each time we encounter a successful mutation and resets each time we encounter stale population.<br>
        Significant progress toward the fitness function's optimum, generally, can only happen in a narrow band of the mutation step size sigma. That narrow band is called evolution window.<br>
        There are three well-known methods to adapt the mutation step size σ in evolution strategies:<br>
            <li>(1/5-th) Success Rule</li>
            <li>Self-Adaptation (for example through log-normal mutations)</li>
            <li>Cumulative Step Size Adaptation (CSA)</li>
    The evolution strategy utilized in this project is the 1/5-th success rule.<br>
    <b>Note:</b> Eventually I didn't use this method since the initialization and cross over methods where enough for the algorithm to function correctly and it didn't change the runtime significantly.
    </p>
    <h4>Handling Stale Population:</h4>
    <p>
        To detect a stale population, we have a counter which counts each time we see that the two best chromosomes have the same fitness. If this counter reaches 100, we reset the algorithm with new initial population and the base mutation rate.
    </p>
</p>

In [1]:
def read_file(filename : str) -> list() :
    file = open(filename, 'r+')
    init_board = []
    for line in file.readlines():
        init_board.append([int(i) for i in line.split()])
        
    return init_board

def print_chromosome(chromosome):
    if chromosome == None:
        return
    for i in range(len(chromosome)):
        for j in range(len(chromosome[0])):
            print(chromosome[i][j], end = " ")
        print()
    print()    


Some utility functions for checking rule violations in the board and crossing over.

In [2]:
def is_row_duplicate(row, value, chromosome):
    for column in range(0, 9):
        if chromosome[row][column] == value:
            return True
    return False

def is_column_duplicate(column, value, chromosome):
    for row in range(0, 9):
        if chromosome[row][column] == value:
            return True
    return False

def is_block_duplicate(row, column, value, chromosome):
    i = 3 * (int(row / 3))
    j = 3 * (int(column / 3))

    if ((chromosome[i][j] == value)
    or (chromosome[i][j + 1] == value)
    or (chromosome[i][j + 2] == value)
    or (chromosome[i + 1][j] == value)
    or (chromosome[i + 1][j + 1] == value)
    or (chromosome[i + 1][j + 2] == value)
    or (chromosome[i + 2][j] == value)
    or (chromosome[i + 2][j + 1] == value)
    or (chromosome[i + 2][j + 2] == value)):
        return True
    return False

def find_unused(parent_row, remaining):
    for i in range(0, len(parent_row)):
        if (parent_row[i] in remaining):
            return i

def find_value(parent_row, value):
    for i in range(0, len(parent_row)):
        if (parent_row[i] == value):
            return i

Utility function for choosing the parents for cross over, this function gives the weaker chromosomes a chance to be chosen rather than being removed forever like my first cross over algorithm. This is useful for decreasing the chance of getting stuck in local maxima since there are possible solutions in the weaker chromosmes and possible local maximas in fitter chromosomes.

In [3]:
def compete(chromosome1, chromosome2):

    fittest = chromosome2[1]
    weakest = chromosome1[1]

    if (chromosome1[0] > chromosome2[0]):
        fittest, weakest = weakest, fittest

    selection_rate = 0.7
    r = random.uniform(0, 1.1)
    while (r > 1):  # Outside [0, 1] boundary. Choose another.
        r = random.uniform(0, 1.1)
    if (r < selection_rate):
        return fittest
    return weakest

In [4]:
import random
from copy import deepcopy
from math import floor
import numpy as np

random.seed()

class SudokuSolver():

    def __init__(self, init_board):
        self.fixed_values = init_board
        self.solved = 0
        self.pop_size = 200  # population size
        self.elite_percentage = 10 # keep x% best of parents in the new population
        self.Ng = 10000  # Number of generations.
        self.m_count = 0  # Number of mutations.
        self.ans = []
        
        # Mutation parameters.
        self.pm = 0.06

        # Create an initial population.
        self.chromosomes = self.generate_initial_population()
        return
    
    def generate_initial_population(self):
        self.stale = 0
        init_pop = []
        
        # Determine the legal values that each cell can take.
        helper = [[[] for j in range(0, 9)] for i in range(0, 9)]
        
        for row in range(0, 9):
            for column in range(0, 9):
                for value in range(1, 10):
                    if ((self.fixed_values[row][column] == 0) and not 
                        (is_column_duplicate(column, value, self.fixed_values) or 
                         is_block_duplicate(row, column, value, self.fixed_values) or 
                         is_row_duplicate(row, value, self.fixed_values))):
                        # Value is available.
                        helper[row][column].append(value)
                    elif self.fixed_values[row][column] != 0:
                        helper[row][column].append(self.fixed_values[row][column])
                        break

        for p in range(0, self.pop_size):
            chromosome = []
            for i in range(0, 9):
                row = [0]*9
                
                # There must be no duplicates in the row.
                while len(list(set(row))) != 9:
                    for j in range(0, 9):
                        if self.fixed_values[i][j] == 0:
                            row[j] = helper[i][j][random.randint(0, len(helper[i][j]) - 1)]
                        else:
                            row[j] = self.fixed_values[i][j]

                chromosome.append(row)
            init_pop.append(chromosome)

        print("Initial fitness:", self.get_fitness(init_pop[0]))
        return init_pop
    
    def cross_over(self, chromosome1, chromosome2):
        # Make a copy of the parent genes.
        child1 = deepcopy(chromosome1)
        child2 = deepcopy(chromosome2)

        # Pick two crossover points. Crossover must have at least 1 row (and at most 8) rows.
        crossover_point1 = random.randint(0, 8)
        crossover_point2 = random.randint(1, 9)
        while (crossover_point1 == crossover_point2):
            crossover_point1 = random.randint(0, 8)
            crossover_point2 = random.randint(1, 9)

        if (crossover_point1 > crossover_point2):
            crossover_point1, crossover_point2 = crossover_point2, crossover_point1
            
        for i in range(crossover_point1, crossover_point2):
            child1[i], child2[i] = self.crossover_rows(child1[i], child2[i])

        return child1, child2
                    
    def crossover_rows(self, row1, row2):
        child_row1 = [0]*9
        child_row2 = [0]*9

        remaining = [i for i in range(1, 10)]
        cycle = 0

        while ((0 in child_row1) and (0 in child_row2)):
            index = find_unused(row1, remaining)
            start = row1[index]
            remaining.remove(row1[index])
            
            if (cycle % 2 == 0):  # Even cycles.
                child_row1[index] = row1[index]
                child_row2[index] = row2[index]
                next = row2[index]

                while (next != start):  # While cycle not done...
                    index = find_value(row1, next)
                    child_row1[index] = row1[index]
                    remaining.remove(row1[index])
                    child_row2[index] = row2[index]
                    next = row2[index]

            else:  # Odd cycle - flip values.
                child_row1[index] = row2[index]
                child_row2[index] = row1[index]
                next = row2[index]

                while (next != start):  # While cycle not done...
                    index = find_value(row1, next)
                    child_row1[index] = row2[index]
                    remaining.remove(row1[index])
                    child_row2[index] = row1[index]
                    next = row2[index]

            cycle += 1

        return child_row1, child_row2

    def mutate(self, chromosome):
        prob = random.random()
        if (prob < self.pm):  # Mutate.
            row1, col1 = (random.randint(0, 8), random.randint(0, 8))
            row2, col2 = (random.randint(0, 8), random.randint(0, 8))

            while (row1, col1) == (row2, col2):
                row1, col1 = (random.randint(0, 8), random.randint(0, 8))
                row2, col2 = (random.randint(0, 8), random.randint(0, 8))
            row2 = row1

            # Check if the two places are not fixed
            if self.fixed_values[row1][col1] == 0 and self.fixed_values[row2][col2] == 0:
                temp_chromosome = deepcopy(chromosome)
                temp = temp_chromosome[row2][col2]
                temp_chromosome[row2][col2] = temp_chromosome[row1][col1]
                temp_chromosome[row1][col1] = temp
                # and we are not lowering fitness.
                if self.get_fitness(temp_chromosome) > self.get_fitness(chromosome):
                    chromosome = deepcopy(temp_chromosome)

        return chromosome
        
        
    def get_fitness(self, chromosome):
        row_count = [0]*9
        column_count = [0]*9
        block_count = [0]*9
        row_sum = 0
        column_sum = 0
        block_sum = 0
        
        # For each column....
        for j in range(0, 9):
            for i in range(0, 9):
                column_count[chromosome[i][j] - 1] += 1
                
            for i in range(0, 9):
                row_count[chromosome[j][i] - 1] += 1

            for k in range(9):
                if column_count[k] == 1:
                    column_sum += 1
                if row_count[k] == 1:
                    row_sum += 1
            column_count = [0]*9
            row_count = [0]*9

        # For each block...
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                block_count[chromosome[i][j] - 1] += 1
                block_count[chromosome[i][j + 1] - 1] += 1
                block_count[chromosome[i][j + 2] - 1] += 1

                block_count[chromosome[i + 1][j] - 1] += 1
                block_count[chromosome[i + 1][j + 1] - 1] += 1
                block_count[chromosome[i + 1][j + 2] - 1] += 1

                block_count[chromosome[i + 2][j] - 1] += 1
                block_count[chromosome[i + 2][j + 1] - 1] += 1
                block_count[chromosome[i + 2][j + 2] - 1] += 1

                for k in range(9):
                    if block_count[k] == 1:
                        block_sum += 1
                block_count = [0]*9

        return column_sum + block_sum + row_sum

    
    def generate_new_population(self, generation):
        sorted_chromosomes = [(self.get_fitness(chromosome), chromosome) for chromosome in self.chromosomes]
        sorted_chromosomes.sort(reverse=True)
        
        # Check for stale population.
        if (sorted_chromosomes[0] != sorted_chromosomes[1]):
            self.stale = 0
        else:
            self.stale += 1

        # Re-seed the population if 100 generations have passed
        # with the fittest two candidates always having the same fitness.
        if (self.stale >= 100):
            print("The population has gone stale. Re-seeding...")
            return self.generate_initial_population()

        elites = sorted_chromosomes[:floor((self.elite_percentage / 100) * (self.pop_size))]
        print("Generation {}. Best fitness: {}. Second best: {}".format(generation, elites[0][0], elites[1][0]))
        if (sorted_chromosomes[0][0] == 243): #no intersections in the best chromosome
            self.solved = 1
            self.ans = sorted_chromosomes[0][1]
            return sorted_chromosomes[0]

        new_pop = [chromosome[1] for chromosome in elites] # add elites

        # Create the rest of the candidates.
        while (len(new_pop) < self.pop_size):
            # Select parents from population via a tournament.
            potential_parents = random.sample(sorted_chromosomes, 4)
            parent1 = compete(potential_parents[0], potential_parents[1])
            parent2 = compete(potential_parents[2], potential_parents[3])

            child1, child2 = self.cross_over(parent1, parent2)

            mutated_child1 = self.mutate(child1)
            mutated_child2 = self.mutate(child2)
            new_pop.append(mutated_child1)
            new_pop.append(mutated_child2)

        return new_pop

    def solve(self):        
        for generation in range(0, self.Ng):
            # Create the next population.
            self.chromosomes = self.generate_new_population(generation)
            if self.solved:
                return self.ans

        print("No solution found.")
        return None

In [5]:
from time import time


sudoku_solver = SudokuSolver(read_file("Test1.txt"))

start = time()

print_chromosome(sudoku_solver.solve())

end = time()

print("Time taken:", (end-start))

Initial fitness: 202
Generation 0. Best fitness: 219. Second best: 218
Generation 1. Best fitness: 219. Second best: 218
Generation 2. Best fitness: 219. Second best: 218
Generation 3. Best fitness: 221. Second best: 220
Generation 4. Best fitness: 225. Second best: 224
Generation 5. Best fitness: 225. Second best: 225
Generation 6. Best fitness: 225. Second best: 225
Generation 7. Best fitness: 225. Second best: 225
Generation 8. Best fitness: 228. Second best: 227
Generation 9. Best fitness: 231. Second best: 229
Generation 10. Best fitness: 235. Second best: 231
Generation 11. Best fitness: 235. Second best: 235
Generation 12. Best fitness: 235. Second best: 235
Generation 13. Best fitness: 235. Second best: 235
Generation 14. Best fitness: 235. Second best: 235
Generation 15. Best fitness: 235. Second best: 235
Generation 16. Best fitness: 235. Second best: 235
Generation 17. Best fitness: 235. Second best: 235
Generation 18. Best fitness: 235. Second best: 235
Generation 19. Best 

Generation 160. Best fitness: 239. Second best: 239
Generation 161. Best fitness: 239. Second best: 239
The population has gone stale. Re-seeding...
Initial fitness: 203
Generation 163. Best fitness: 219. Second best: 216
Generation 164. Best fitness: 219. Second best: 216
Generation 165. Best fitness: 220. Second best: 219
Generation 166. Best fitness: 229. Second best: 222
Generation 167. Best fitness: 231. Second best: 229
Generation 168. Best fitness: 233. Second best: 231
Generation 169. Best fitness: 233. Second best: 233
Generation 170. Best fitness: 233. Second best: 233
Generation 171. Best fitness: 233. Second best: 233
Generation 172. Best fitness: 237. Second best: 233
Generation 173. Best fitness: 237. Second best: 235
Generation 174. Best fitness: 237. Second best: 235
Generation 175. Best fitness: 237. Second best: 237
Generation 176. Best fitness: 239. Second best: 237
Generation 177. Best fitness: 239. Second best: 239
Generation 178. Best fitness: 243. Second best: 24

In [6]:
sudoku_solver = SudokuSolver(read_file("Test2.txt"))

start = time()

print_chromosome(sudoku_solver.solve())

end = time()

print("Time taken:", (end-start))

Initial fitness: 198
Generation 0. Best fitness: 214. Second best: 210
Generation 1. Best fitness: 214. Second best: 210
Generation 2. Best fitness: 217. Second best: 214
Generation 3. Best fitness: 217. Second best: 217
Generation 4. Best fitness: 219. Second best: 217
Generation 5. Best fitness: 221. Second best: 219
Generation 6. Best fitness: 229. Second best: 221
Generation 7. Best fitness: 229. Second best: 229
Generation 8. Best fitness: 229. Second best: 229
Generation 9. Best fitness: 229. Second best: 229
Generation 10. Best fitness: 231. Second best: 229
Generation 11. Best fitness: 237. Second best: 235
Generation 12. Best fitness: 237. Second best: 237
Generation 13. Best fitness: 237. Second best: 237
Generation 14. Best fitness: 239. Second best: 237
Generation 15. Best fitness: 239. Second best: 239
Generation 16. Best fitness: 239. Second best: 239
Generation 17. Best fitness: 239. Second best: 239
Generation 18. Best fitness: 239. Second best: 239
Generation 19. Best 

This algorithm can be improved using adaptive mutation rate and better cross over strategy.<br>
The initialization takes a long time since it tries to limit the intersections to rows and blocks. This initialization can be random while other sections of the algorithms are improved.<br>
To prevent the population from going stale and getting stuck in local maxima we can decrease the fitness of the elites who have stayed in the opulation for more than generation.<br>
Based on my experiments with the algorithm, the fitness function doesn't really affect the algorithms efficiency.<br>
My code was inspired by: https://github.com/chinyan/Genetic-Algorithm-based-Sudoku-Solver<br>
There were also several articles regarding the algorithm and the problem which helped me understand the problem and find a better solution:<br>
<li>Solving, rating and generating Sudoku puzzles with GA-October 2007
<li>Evolution Strategies-Nikolaus Hansen, Dirk V. Arnold and Anne Auger-February 11, 2015
